In [16]:
# Instalar compilador y OpenCV C++
!apt-get update
!apt-get install -y g++ pkg-config libopencv-dev

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,594 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,838 kB]
Fetched 5,816 kB in 1s (4,277 kB

In [17]:
from google.colab import files
uploaded = files.upload()

In [18]:
%%writefile highboost_laplacian.cpp

#include <opencv2/opencv.hpp>
#include <iostream>

using namespace cv;
using namespace std;

int main(int argc, char** argv) {
    if (argc < 4) {
        cout << "Uso: " << argv[0] << " <imagen> <A> <salida>\n";
        cout << "Ejemplo: " << argv[0] << " lena.png 2.0 salida.png\n";
        return 1;
    }

    string imagePath = argv[1];
    double A = stod(argv[2]);
    string outPath = argv[3];

    // Leer imagen y convertir a escala de grises
    Mat img = imread(imagePath, IMREAD_GRAYSCALE);
    if (img.empty()) {
        cerr << "Error: no se pudo abrir la imagen " << imagePath << endl;
        return -1;
    }

    Mat imgF;
    img.convertTo(imgF, CV_32F);

    // 1) Definir kernels
    Mat W_allpass = (Mat_<float>(3,3) <<
                      0, 0, 0,
                      0, 1, 0,
                      0, 0, 0);

    Mat W_highpass = (Mat_<float>(3,3) <<
                      0, -1, 0,
                     -1,  4, -1,
                      0, -1, 0);

    // 2) Calcular el kernel High-Boost
    Mat W_highboost = A * W_allpass + W_highpass;

    cout << "Kernel High-Boost (A = " << A << "):" << endl;
    cout << W_highboost << endl;

    // 3) Aplicar filtro
    Mat result;
    filter2D(imgF, result, -1, W_highboost, Point(-1,-1), 0, BORDER_REPLICATE);

    // 4) Normalizar y convertir
    normalize(result, result, 0, 255, NORM_MINMAX);
    result.convertTo(result, CV_8U);

    // 5) Guardar resultado
    imwrite(outPath, result);

    imwrite("original_temp.png", img);
    imwrite("resultado_temp.png", result);

    cout << "Resultado guardado en " << outPath << endl;

    return 0;
}


Overwriting highboost_laplacian.cpp


In [19]:
# Compilar
!g++ highboost_laplacian.cpp -o highboost `pkg-config --cflags --libs opencv4`
# Ejecutar
!./highboost g.png 1.5 salida.png


Kernel High-Boost (A = 1.5):
[0, -1, 0;
 -1, 5.5, -1;
 0, -1, 0]
Resultado guardado en salida.png
